In [31]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pandas as pd

urls = []

# 最後一頁:https://www.mobile01.com/topiclist.php?f=291&p=3510

def get_urls():
    for i in range(1,3510):
        try:
            res = requests.get(f'https://www.mobile01.com/topiclist.php?f=291&p={i}', headers={'User-Agent': 'Mozilla/5.0'})
            soup = BeautifulSoup(res.text,'html.parser')
            a_s = soup.select('div.c-listTableTd__title > a')
            for a in a_s:
                if re.match('^topicdetail',a.get('href')):
                    urls.append(a.get('href'))
            
#            break
        except:
            print('Error')
            print(f'https://www.mobile01.com/topiclist.php?f=291&p={i}')
            break


url_list = []
title_list = []
author_list = []
date_list = []
content_list = []

def get_content():

    for url in urls:
        try:
            # mobile01需要加header
            res = requests.get('https://www.mobile01.com/'+url, headers={'User-Agent': 'Mozilla/5.0'})
            soup = BeautifulSoup(res.text,'html.parser')
            url_list.append(url)

            try:
                date_list.append(soup.select_one('div.l-navigation__item.is-dockingHide > ul > li > span').text)
            except:
                date_list.append('')
            try:
                title_list.append(soup.select_one('h1.t2').text)
            except:
                title_list.append('')
            try:
                author_list.append(soup.select_one('div.c-authorInfo__id > a').text)
            except:
                author_list.append('')
            try:
                content_list.append(soup.select_one('article > div').text)
            except:
                content_list.append('')



        except:
            print(url)

def save_csv():            
    pd_store= pd.DataFrame(columns = ['URL','Date','Title','Author','Content'])
    pd_store['URL'] = url_list
    pd_store['Date'] = date_list
    pd_store['Title'] = title_list
    pd_store['Author'] = author_list
    pd_store['Content'] = content_list

    pd_store.to_csv('mobile01_2.csv', index=False)
    
# main
get_urls()
get_content()
save_csv()